In [ ]:
import os
import holidays 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import pandas_ta as ta
import numpy as np
from statsmodels.tsa.stattools import adfuller

data = '2012-12-30'

path_stock = "../data/stock"
path_fed = "../data/fed"

# Top 10 Tech Companies based on Market Cap

In [ ]:
AAPL_df = pd.read_csv(f"{path_stock}/AAPL_stock.csv")
MSFT_df = pd.read_csv(f"{path_stock}/MSFT_stock.csv")
GOOGL_df = pd.read_csv(f"{path_stock}/GOOGL_stock.csv")
NVDA_df = pd.read_csv(f"{path_stock}/NVDA_stock.csv")
AMZN_df = pd.read_csv(f"{path_stock}/AMZN_stock.csv")
META_df = pd.read_csv(f"{path_stock}/META_stock.csv")
TSLA_df = pd.read_csv(f"{path_stock}/TSLA_stock.csv")
AVGO_df = pd.read_csv(f"{path_stock}/AVGO_stock.csv")
AMD_df = pd.read_csv(f"{path_stock}/AMD_stock.csv")
CRM_df = pd.read_csv(f"{path_stock}/CRM_stock.csv")

In [ ]:
['date', 'close_AAPL', 'high_AAPL', 'low_AAPL', 'open_AAPL',
       'volume_AAPL', 'delta_price_AAPL', 'avg_price_AAPL', 'price_ratio_AAPL',
       'invest_AAPL']
['date', 'close_MSFT', 'high_MSFT', 'low_MSFT', 'open_MSFT',
       'volume_MSFT', 'delta_price_MSFT', 'avg_price_MSFT', 'price_ratio_MSFT',
       'invest_MSFT']
['date', 'close_GOOGL', 'high_GOOGL', 'low_GOOGL', 'open_GOOGL',
       'volume_GOOGL', 'delta_price_GOOGL', 'avg_price_GOOGL',
       'price_ratio_GOOGL', 'invest_GOOGL']
['date', 'close_NVDA', 'high_NVDA', 'low_NVDA', 'open_NVDA',
       'volume_NVDA', 'delta_price_NVDA', 'avg_price_NVDA', 'price_ratio_NVDA',
       'invest_NVDA']
['date', 'close_AMZN', 'high_AMZN', 'low_AMZN', 'open_AMZN',
       'volume_AMZN', 'delta_price_AMZN', 'avg_price_AMZN', 'price_ratio_AMZN',
       'invest_AMZN']
['date', 'close_META', 'high_META', 'low_META', 'open_META',
       'volume_META', 'delta_price_META', 'avg_price_META', 'price_ratio_META',
       'invest_META']
['date', 'close_TSLA', 'high_TSLA', 'low_TSLA', 'open_TSLA',
       'volume_TSLA', 'delta_price_TSLA', 'avg_price_TSLA', 'price_ratio_TSLA',
       'invest_TSLA']
['date', 'close_AVGO', 'high_AVGO', 'low_AVGO', 'open_AVGO',
       'volume_AVGO', 'delta_price_AVGO', 'avg_price_AVGO', 'price_ratio_AVGO',
       'invest_AVGO']
['date', 'close_AMD', 'high_AMD', 'low_AMD', 'open_AMD', 'volume_AMD',
       'delta_price_AMD', 'avg_price_AMD', 'price_ratio_AMD', 'invest_AMD']

['date', 'close_CRM', 'high_CRM', 'low_CRM', 'open_CRM', 'volume_CRM',
       'delta_price_CRM', 'avg_price_CRM', 'price_ratio_CRM', 'invest_CRM']


## Tach companies stock Data Frame processing
- Remove the null / header
- Make some features Engineering
- Change the column name
- Change the time type

In [ ]:
import pandas as pd

def process_stock_data(df, ticker_symbol):
    """
    Processes a stock data DataFrame.

    Args:
        df (pd.DataFrame): DataFrame with stock data (Price, Close, High, Low, Open, Volume, Ticker).
        ticker_symbol (str): Stock ticker symbol (e.g., 'AAPL').

    Returns:
        pd.DataFrame: Processed DataFrame with calculated features and renamed columns.
    """
    df.dropna(inplace=True)  # Remove rows containing any missing values.

    columns_to_convert = ['Close', 'High', 'Low', 'Open', 'Volume']
    df[columns_to_convert] = df[columns_to_convert].astype(float)  # Convert specified price/volume columns to floating-point numbers.

    # Calculate new features based on price data:
    df["delta_price"] = df["High"] - df["Low"]  # Calculate the difference between the high and low price for each day.
    df["avg_price"] = (df["Close"] + df["High"] + df["Low"] + df["Open"]) / 4  # Calculate the average of the close, high, low, and open prices.
    df["price_ratio"] = df["delta_price"] / df["avg_price"]  # Calculate the ratio of the delta price to the average price.
    df["invest"] = df["Volume"] * df["avg_price"]  # Calculate the difference between the trading volume and the average price (note: this might not be a standard financial metric and could be re-evaluated).

    # Rename the columns for clarity and to include the ticker symbol:
    df.rename(columns={"Price": "date",  # Rename the 'Price' column to 'date'.
                        "Close": f"close_{ticker_symbol}",  # Rename 'Close' to 'cl_ticker'.
                        "High": f"high_{ticker_symbol}",  # Rename 'High' to 'hi_ticker'.
                        "Low": f"low_{ticker_symbol}",  # Rename 'Low' to 'lo_ticker'.
                        "Open": f"open_{ticker_symbol}",  # Rename 'Open' to 'op_ticker'.
                        "delta_price": f"delta_price_{ticker_symbol}",  # Rename 'delta_price' to 'de_ticker'.
                        "avg_price": f"avg_price_{ticker_symbol}",  # Rename 'avg_price' to 'av_ticker'.
                        "invest": f"invest_{ticker_symbol}",  # Rename 'invest' to 'va_ticker'.
                        "price_ratio": f"price_ratio_{ticker_symbol}",  # Rename 'ratio' to 'ra_ticker'.
                        'Volume': f'volume_{ticker_symbol}'}, inplace=True)  # Rename 'Volume' to 'Vo_ticker'.

    df['date'] = pd.to_datetime(df['date'])  # Convert the 'date' column to datetime objects for proper time series handling.

    df.reset_index(drop=True, inplace=True)  # Reset the DataFrame's index to a default integer index and drop the original index.

    # Drop the 'Ticker' column as the ticker information is now embedded in the column names:
    if 'Ticker' in df.columns:
        df.drop('Ticker', axis=1, inplace=True)

    return df  # Return the processed DataFrame.

### Tech companies stock clean Data Frame 

In [ ]:
AAPL_clean_df = process_stock_data(AAPL_df, 'AAPL')
MSFT_clean_df = process_stock_data(MSFT_df, 'MSFT')
GOOGL_clean_df = process_stock_data(GOOGL_df, 'GOOGL')
NVDA_clean_df = process_stock_data(NVDA_df, 'NVDA')
AMZN_clean_df = process_stock_data(AMZN_df, 'AMZN')
META_clean_df = process_stock_data(META_df, 'META')
TSLA_clean_df = process_stock_data(TSLA_df, 'TSLA')
AVGO_clean_df = process_stock_data(AVGO_df, 'AVGO')
AMD_clean_df = process_stock_data(AMD_df, 'AMD')
CRM_clean_df = process_stock_data(CRM_df, 'CRM')

###  Find the Max and Min od Data column in each companies stock Data Frame

In [ ]:
stock_data_ranges = {}

dataframes = {
    "AAPL": AAPL_clean_df,
    "MSFT": MSFT_clean_df,
    "GOOGL": GOOGL_clean_df,
    "NVDA": NVDA_clean_df,
    "AMZN": AMZN_clean_df,
    "META": META_clean_df,
    "TSLA": TSLA_clean_df,
    "AVGO": AVGO_clean_df,
    "AMD": AMD_clean_df,
    "CRM": CRM_clean_df,
}

for name, df in dataframes.items():
    if 'date' in df.columns:
        min_date = df['date'].min()
        max_date = df['date'].max()
        stock_data_ranges[name] = {'min_date': min_date, 'max_date': max_date}
    else:
        print(f"Warning: 'date' column not found in {name}_clean_df")

# Create a Pandas DataFrame to display the results
date_range_df = pd.DataFrame.from_dict(stock_data_ranges, orient='index')
date_range_df.index.name = 'Stock'

print(date_range_df)

In above result , It seems that the META is started from 2012 while almost the others started from 2000.

## Macro Indicators from Yahoo Finance:
- Indices
- Commodities
- Sector ETFs (Proxies)
- Other Market Metrics

In [ ]:
macro_df = pd.read_csv(f"{path_stock}/macro_indicators_full.csv")
# Convert the 'date' column to datetime objects
macro_df['Date'] = pd.to_datetime(macro_df['Date'])
macro_df.rename(columns={"Date": "date"}, inplace=True)

In [ ]:
macro_df.isnull().sum()

Data Frame : macro_df ---> Has some missing values that need to be check according to the time. 

Let Filter the time after the '2012-05-31'. This is exactly after the time which we have the META stock data frame. 

In [ ]:
macro_df_filter = macro_df[macro_df['date'] > data ]
min_date_macro_df_filter = macro_df_filter['date'].min()
max_date_macro_df_filter = macro_df_filter['date'].max()
macro_df_filter.isnull().sum()

In [ ]:
#macro_df_filter = macro_df_filter.drop('Brent_Crude_Futures',axis=1)

In [ ]:
df = macro_df_filter
data_name = 'macro_df_filter'
# 1. Matrix Plot: Visualize the pattern of missingness
plt.figure(figsize=(10, 6))
msno.matrix(df)
plt.title(f'Missing Value Matrix - {data_name}')
plt.show()

In [ ]:
macro_clean_df = macro_df_filter.dropna()
macro_clean_df.isnull().sum()

# Fed Data frame

In [ ]:
fed_df = pd.read_csv(f"{path_fed}/combined_economic_indicators.csv")

# Rename the 'Unnamed: 0' column to 'date'
fed_df.rename(columns={'Unnamed: 0': 'date'}, inplace=True)

# Convert the 'date' column to datetime objects
fed_df['date'] = pd.to_datetime(fed_df['date'])

In [ ]:
fed_df.isnull().sum()

In [ ]:
fed_df_filter = fed_df[fed_df['date'] > data]
min_date_fed_df_filter = fed_df_filter['date'].min()
max_date_fed_df_filter = fed_df_filter['date'].max()
fed_df_filter.isnull().sum()

In [ ]:
df = fed_df_filter
data_name = 'fed_df_filter'
# 1. Matrix Plot: Visualize the pattern of missingness
plt.figure(figsize=(10, 6))
msno.matrix(df)
plt.title(f'Missing Value Matrix - {data_name}')
plt.show()

In [ ]:
fed_clean_df = fed_df_filter[['date', 'cpi', 'fed_rate', 'consumer_confidence','vix', 'oil', 'nonfarm_payrolls',
       'treasury_yield', 'industrial_production', 'retail_sales', 'pmi',
        'day_of_week', 'is_holiday', 'is_working_day']].dropna()
fed_clean_df.isnull().sum()

# Merging the Date Frames

In [ ]:
# Start with the first DataFrame
merged_stock_data = AAPL_clean_df.copy()

# List of stock DataFrames (excluding the first one)
stock_dfs = [MSFT_clean_df, GOOGL_clean_df, NVDA_clean_df, AMZN_clean_df,
             META_clean_df, TSLA_clean_df, AVGO_clean_df, AMD_clean_df, CRM_clean_df]

# Merge each stock DataFrame on 'date' using a left join
for df in stock_dfs:
    merged_stock_data = pd.merge(merged_stock_data, df, on='date', how='inner')

# 2. Merge with Macro and Fed DataFrames

# Merge stock data with macro data
merged_data = pd.merge(merged_stock_data, macro_clean_df, on='date', how='inner')

# Merge with fed data
final_merged_df = pd.merge(merged_data, fed_clean_df, on='date', how='inner')

stock_df = merged_stock_data
stock_macro_df = merged_data
stock_macro_fed_df = final_merged_df

# Feature Engineering

In [ ]:
# The total of trade for the top 10 stock companies per day and their percentage
stock_macro_fed_df["invest_total"] = (stock_macro_fed_df["invest_AAPL"] +
                                stock_macro_fed_df["invest_MSFT"] +
                                stock_macro_fed_df["invest_GOOGL"] +
                                stock_macro_fed_df["invest_NVDA"] +
                                stock_macro_fed_df["invest_AMZN"] +
                                stock_macro_fed_df["invest_META"] +
                                stock_macro_fed_df["invest_TSLA"] +
                                stock_macro_fed_df["invest_AVGO"] +
                                 stock_macro_fed_df["invest_AMD"] +
                                stock_macro_fed_df["invest_CRM"] )

stock_macro_fed_df["invest_AAPL_ratio"] = stock_macro_fed_df["invest_AAPL"]/stock_macro_fed_df["invest_total"]
stock_macro_fed_df["invest_MSFT_ratio"] = stock_macro_fed_df["invest_MSFT"] /stock_macro_fed_df["invest_total"]
stock_macro_fed_df["invest_GOOGL_ratio"] = stock_macro_fed_df["invest_GOOGL"] /stock_macro_fed_df["invest_total"]
stock_macro_fed_df["invest_NVDA_ratio"] = stock_macro_fed_df["invest_NVDA"] /stock_macro_fed_df["invest_total"]
stock_macro_fed_df["invest_AMZN_ratio"] = stock_macro_fed_df["invest_AMZN"] /stock_macro_fed_df["invest_total"]
stock_macro_fed_df["invest_META_ratio"] = stock_macro_fed_df["invest_META"] /stock_macro_fed_df["invest_total"]
stock_macro_fed_df["invest_TSLA_ratio"] = stock_macro_fed_df["invest_TSLA"] /stock_macro_fed_df["invest_total"]
stock_macro_fed_df["invest_AVGO_ratio"] = stock_macro_fed_df["invest_AVGO"] /stock_macro_fed_df["invest_total"]
stock_macro_fed_df["invest_AMD_ratio"] = stock_macro_fed_df["invest_AMD"] /stock_macro_fed_df["invest_total"]
stock_macro_fed_df["invest_CRM_ratio"] = stock_macro_fed_df["invest_CRM"] /stock_macro_fed_df["invest_total"]

In [ ]:
# time extraction 
stock_macro_fed_df['day_of_week'] = stock_macro_fed_df['date'].dt.dayofweek       # 0 = Monday
stock_macro_fed_df['month'] = stock_macro_fed_df['date'].dt.month                 # 1 = January
stock_macro_fed_df['week_number'] = stock_macro_fed_df['date'].dt.isocalendar().week  # ISO week number
stock_macro_fed_df['is_month_end'] = stock_macro_fed_df['date'].dt.is_month_end.astype(int)  # 1 if end of month, else 0

In [ ]:
# 📦 Feature Engineering and Stationarization Script
# --- Setup ---
df = stock_macro_fed_df.copy()
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# --- Macroeconomic, Indices, ETFs to be differenced ---
macro_and_indices_cols = [
    'cpi', 'fed_rate', 'consumer_confidence', 'vix', 'oil',
    'nonfarm_payrolls', 'treasury_yield', 'industrial_production',
    'retail_sales', 'pmi',
    'S&P500_Index', 'Dow_Jones_Index', 'NASDAQ_Composite',
    'Russell2000_Index', 'VIX_Index', 'Dollar_Index_DXY', 'Gold_Futures',
    'WTI_Oil_Futures', 'Copper_Futures', 'Brent_Crude_Futures',
    'Tech_Sector_ETF', 'Energy_Sector_ETF', 'Financial_Sector_ETF',
    'ConsumerDiscretionary_ETF', 'Lithium_ETF', 'Semiconductor_ETF', 'Electricity_Proxy'
]

# --- Apply first differencing to macroeconomic and index columns ---
for col in macro_and_indices_cols:
    if col in df.columns:
        df[f'{col}_diff'] = df[col].diff()

# --- Stock Tickers ---
stocks = ['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'AMZN', 'META', 'TSLA', 'AVGO', 'AMD', 'CRM']

# --- Feature Engineering for Stock Prices ---
for stock in stocks:
    for field in ['close', 'open', 'high', 'low']:
        col = f'{field}_{stock}'
        if col in df.columns:
            # Differencing to make stationary
            df[f'{col}_diff'] = df[col].diff()
            # Rolling means and std devs
            df[f'{col}_rolling_mean_5'] = df[col].rolling(window=5).mean()
            df[f'{col}_rolling_std_5'] = df[col].rolling(window=5).std()
            df[f'{col}_rolling_mean_20'] = df[col].rolling(window=20).mean()
            df[f'{col}_rolling_std_20'] = df[col].rolling(window=20).std()
            # Lags
            for lag in [1, 3, 5, 10]:
                df[f'{col}_lag_{lag}'] = df[col].shift(lag)

    # --- Technical Indicators (on close prices) ---
    close_col = f'close_{stock}'
    if close_col in df.columns:
        df[f'{stock}_RSI'] = ta.rsi(df[close_col], length=14)
        macd = ta.macd(df[close_col])
        if macd is not None:
            df[f'{stock}_MACD'] = macd['MACD_12_26_9']
            df[f'{stock}_MACD_signal'] = macd['MACDs_12_26_9']
            df[f'{stock}_MACD_hist'] = macd['MACDh_12_26_9']

    # --- Volume Features ---
    vol_col = f'volume_{stock}'
    if vol_col in df.columns:
        df[f'{vol_col}_log'] = np.log1p(df[vol_col])
        df[f'{vol_col}_diff'] = df[f'{vol_col}_log'].diff()

    # --- Delta, Avg, Price Ratio, Invest Columns ---
    for suffix in ['delta_price', 'avg_price', 'price_ratio', 'invest']:
        derived_col = f'{suffix}_{stock}'
        if derived_col in df.columns:
            df[f'{derived_col}_diff'] = df[derived_col].diff()

# --- Portfolio level investments ---
portfolio_cols = ['invest_total', 'invest_AAPL_ratio', 'invest_MSFT_ratio', 'invest_GOOGL_ratio',
                  'invest_NVDA_ratio', 'invest_AMZN_ratio', 'invest_META_ratio',
                  'invest_TSLA_ratio', 'invest_AVGO_ratio', 'invest_AMD_ratio', 'invest_CRM_ratio']

for col in portfolio_cols:
    if col in df.columns:
        df[f'{col}_diff'] = df[col].diff()

# --- Drop rows with NaNs caused by rolling, diff, or lagging ---
df.dropna(inplace=True)

# Stationarity Testing (ADF) for Time Series Modeling

In [ ]:
# 📈 Stationarity Testing (ADF) for Time Series Modeling
#from statsmodels.tsa.stattools import adfuller
df
# Prepare results list
stationarity_results = []

col_num = len(df.columns)
# Loop over numeric columns and test stationarity
for col in df.select_dtypes(include='number').columns:
    try:
        result = adfuller(df[col].dropna())
        p_value = result[1]
        is_stationary = p_value < (0.05 / col_num)
        stationarity_results.append({
            'Feature': col,
            'ADF Statistic': result[0],
            'p-value': p_value,
            'Critical Value (5%)': result[4]['5%'],
            'Stationary (p < 0.05)': is_stationary
        })
    except Exception as e:
        stationarity_results.append({
            'Feature': col,
            'ADF Statistic': None,
            'p-value': None,
            'Critical Value (5%)': None,
            'Stationary (p < 0.05)': False,
            'Error': str(e)
        })

# Convert results to DataFrame
stationarity_df = pd.DataFrame(stationarity_results)

# Sort by p-value
stationarity_df = stationarity_df.sort_values(by='p-value')


In [ ]:
# Display Stationarity testing result
stationarity_df['Stationary (p < 0.05)'].value_counts()